In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
import tensorflow as tf
import keras
from keras import layers
from tensorflow.keras.layers import LSTM
from keras.layers import Activation, Dense
from tensorflow.keras.models import *
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from tensorflow. keras . callbacks import EarlyStopping
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import datetime as dt

In [ ]:
stock_data= pd.read_csv('Nifty50.csv' ,index_col='Date ')
stock_data.head(3)
stock_data.dropna(inplace=True)

In [ ]:
stock_data.index = pd.to_datetime(stock_data.index)
# Plot the High and Low columns
plt.figure(figsize=(15, 10))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=60))
x_dates = stock_data.index.date

plt.plot(x_dates, stock_data['High'], label='High')
plt.plot(x_dates, stock_data['Low'], label='Low')
plt.xlabel('Time Scale')
plt.ylabel('scaled USD')
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
target_y=stock_data['Close']
X_feat=stock_data.iloc[:,0:3]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_ft=sc.fit_transform(X_feat.values)
X_ft=pd.DataFrame(columns=X_feat.columns,data=X_ft,index=X_feat.index)

In [ ]:
def lstm_split(data,n_steps):
  X,y=[],[]
  for i in range(len(data)-n_steps+1):
    X.append(data[i:i + n_steps,:-1])
    y. append(data[i + n_steps-1,-1])
  return np.array(X), np.array(y)

In [ ]:
X1,y1=lstm_split(stock_data.values, n_steps=2)

train_split=0.8
split_idx=int(np.ceil(len(X1)*train_split))
date_index=stock_data.index

X_train,X_test=X1[:split_idx],X1[split_idx:]
y_train,y_test=y1[:split_idx],y1[split_idx:]
X_train_date,X_test_date=date_index[split_idx:],date_index[split_idx:]
print(X1.shape,X_train.shape,X_test.shape,y_test.shape)

In [ ]:
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(X_train.shape[1], X_train.shape[2]),
              activation='relu',return_sequences=True))
lstm.add(Dense(40,activation='relu'))
lstm.add(Dense(80,activation='relu'))
lstm.add(Dense(40,activation='relu'))
lstm.add(Dense(1))
lstm.compile(loss='mean_absolute_error',optimizer='adam')
lstm.summary()

In [ ]:
history=lstm.fit(X_train,y_train,epochs=500,batch_size=10,verbose=2,shuffle=False)

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
y_pred=lstm.predict(X_test)

In [ ]:
# prompt: create graph of predicted values vs real values
y_pred_r = y_pred[:, 0, 0]
plt.figure(figsize=(15, 10))
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=60))
x_dates = X_test_date
plt.plot(x_dates[:410], y_pred_r[:410], label='Predicted')
plt.plot(x_dates, y_test, label='Real')
plt.xlabel('Time Scale')
plt.ylabel('USD')
plt.legend()
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
date_input = input('Enter a date in YYYY-MM-DD format: ')
date_input = dt.datetime.strptime(date_input, '%Y-%m-%d')
date_index = stock_data.index.get_loc(date_input)
X_input = X_feat.iloc[date_index]
X_input = sc.transform(np.array(X_input).reshape(1, -1))
X_input = np.array(X_input).reshape(1, 2, 3)
y_pred = lstm.predict(X_input)
predicted_price = y_pred[0, 0, 0]
print(f"Predicted price for {date_input}: {predicted_price}")